# Выбор локации для скважины

Нужно решить, где бурить новую скважину.

Предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Необходимо построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Требуется проанализировать возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.
- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

**Легенда:**
- id — уникальный идентификатор скважины;
- f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
- product — объём запасов в скважине (тыс. баррелей).

## Импорт требуемых библиотек ##

In [25]:
import pandas as pd             #программная библиотека для обработки и анализа данных. Работа pandas с данными строится поверх библиотеки NumPy
import numpy as np              #поддержка многомерных массивов; поддержка высокоуровневых математических функций, предназначенных для работы с многомерными массивами

from sklearn.linear_model import LinearRegression      #импорт модуля линейной регрессии

from sklearn.model_selection import train_test_split   #функция train_test_split для разделения датафрейма

from sklearn.metrics import mean_squared_error         #импорт среднеквардатичного отклонения для оценки качества модели

## Загрузка и подготовка данных

In [26]:
df0 = pd.read_csv('/datasets/geo_data_0.csv') #серверный путь
df0.name = 'District 1'

df1 = pd.read_csv('/datasets/geo_data_1.csv') #серверный путь
df1.name = 'District 2'

df2 = pd.read_csv('/datasets/geo_data_2.csv') #серверный путь
df2.name = 'District 3'

In [27]:
display(df0.head(3))
display(df1.head(3))
display(df2.head(3))

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910


In [28]:
df_list = [df0, df1, df2]

for df_ in df_list:
    df_.info()
    print()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Nul

In [29]:
df_list = [df0, df1, df2]

for df_ in df_list:
    display(df_.describe())

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


*Описание данных:*

- 100000 строк, 5 столбцов;
- пропусков и дубликатов нет;
- не требуется изменение имен столбцов;
- не требуется изменение типа данных.

`Целевой признак:`
- product — объём запасов в скважине (тыс. баррелей).

`Признаки:`
- f0,f1, f2 - влияют на объём запасов в скважине.

## Обучение и проверка модели

In [30]:
def production_est(df):
    features = df.drop(columns=['id', 'product'])     #определение признаков
    target = df['product']                 #определение целевого признака

    #деление датафреймов на обучающую и валидационную выборки
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=42)

    model = LinearRegression()                         #загрузка модели линейной регрессии
    model.fit(features_train,target_train)             #обучение модели линейной регрессии
    predictions_valid = model.predict(features_valid)  #предсказания модели на базе валидационной выборки

    #rmse = mean_squared_error(target_valid, predictions_valid)**0.5 #расчет корня из среднеквадратичного отклонения
        
    return target_valid.reset_index(drop=True), pd.Series(predictions_valid)

In [31]:
region_1_target, region_1_predictions = production_est(df0) #сохраним факт и прогноз модели валидационной выборки District 1
region_2_target, region_2_predictions = production_est(df1) #сохраним факт и прогноз модели валидационной выборки District 2
region_3_target, region_3_predictions = production_est(df2) #сохраним факт и прогноз модели валидационной выборки District 3

In [32]:
region_1_predictions.head(3) #посмотрим сформированный прогноз по объему запасов

0    101.901017
1     78.217774
2    115.266901
dtype: float64

In [33]:
region_1_target.head(3) #посмотрим сформированный факт по объему запасов

0    122.073350
1     48.738540
2    131.338088
Name: product, dtype: float64

In [34]:
#расчет корня из среднеквадратичного отклонения

def rmse_procuction_calc(target,predictions):
    rmse = mean_squared_error(target, predictions)**0.5 
    return print(f'- отклонение RMSE = {round(rmse,2)}, запас предсказанного сырья: {round(predictions.mean(),2)}')

In [35]:
print("District #1:")
rmse_procuction_calc(region_1_target, region_1_predictions) #District 1
print("District #2:")
rmse_procuction_calc(region_2_target, region_2_predictions) #District 2
print("District #3:")
rmse_procuction_calc(region_3_target, region_3_predictions) #District 3

District #1:
- отклонение RMSE = 37.76, запас предсказанного сырья: 92.4
District #2:
- отклонение RMSE = 0.89, запас предсказанного сырья: 68.71
District #3:
- отклонение RMSE = 40.15, запас предсказанного сырья: 94.77


**Вывод:**
1. Для региона №1 и №3 корень из среднеквадратичного отклонения по продуктивности находится в одном диапазоне 35-40  по объёму запасов в скважине (тыс. баррелей).
2. У региона №2 отклонение минимальное, меньше 1, что выглядит аномальным: модель идеально предсказывает объем запасов в скважине, сильная корреляция между признаками F0-2 и целевым признаком. Проверим по коээфициенту корреляции Пирсона.

In [36]:
print("коээфициента корреляции Пирсона для всех переменных")
df1.corr()

коээфициента корреляции Пирсона для всех переменных


,f0,f1,f2,product
f0,1.000000,0.182287,-0.001777,-0.030491
f1,0.182287,1.000000,-0.002595,-0.010155
f2,-0.001777,-0.002595,1.000000,0.999397
product,-0.030491,-0.010155,0.999397,1.000000


**Вывод:** Признак f2 и продуктивность / объем запасов скважины имеют положительную линейную зависимость, т.к. коэффициент корреляции равен 0.99 при максимальном значении 1.

## Подготовка к расчёту прибыли

- Бюджет на разработку скважин в регионе — 10 млрд рублей.
- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.

In [37]:
budget = 10000000000                #инвестиции в регион разработки в рублях
barrel_cost = 450                  #доход в рублях с одного барреля
k_revenue_unit = barrel_cost*1000   #перевод дохода на тысяцу баррелей
wells_top = 200                     #топ фокусных скважин
wells_number = 500                  #количество скважин под выборку

In [38]:
budget_revenu_ratio = budget/(k_revenue_unit*wells_top) #объем нефти в скважине для возврата инвестиций в 10 млрд.
print(f'Достаточный объём сырья для безубыточной разработки новой скважины: {round(budget_revenu_ratio,2)} тыс. баррелей.')

Достаточный объём сырья для безубыточной разработки новой скважины: 111.11 тыс. баррелей.


**Вывод:** 
- Объем запасов 1 скважины из 200 в целевом регионе должен быть не менее 111,11 тыс. баррелей, чтобы окупить инвестиции в разработку.
- Расчетный объем запасов превышает средний показатель по District 1 и District 3; кроме того в 1.6 раза больше, чем в среднем для District 2.

## Расчёт прибыли и рисков 

 ### Функция для расчёта прибыли по выбранным скважинам и предсказаниям модели ###


1. Выбрать скважины с максимальными значениями предсказаний (200 шт.).
2. Просуммировать целевое значение объёма сырья, соответствующее этим предсказаниям.
3. Рассчитать прибыль для полученного объёма сырья.

In [39]:
def profit(predictions, target, wells):                            #формула принимает на вход прогноз, факт запасов, количество скважин в фокусе
    predictions_sorted = predictions.sort_values(ascending=False)  #сортируем предсказания в порядке убывания
    target_sorted_top = target[predictions_sorted.index][:wells]   #сортируем факт объема сырья, соответствующее предсказаниям для заданного количества скважин
    profit = target_sorted_top.sum()*k_revenue_unit - budget       #выручка = запасы вы тыс. баррелей * стоимость запасов - инвестиции
    return profit.astype(int)

Проверим, как работает функция на существующей выборке в 200 шт. скважин

In [40]:
profit_1 = profit(region_1_predictions, region_1_target, wells_top)
print("Прибыль по выбранным скважинам District 1:",'{0:,}'.format(profit_1))

Прибыль по выбранным скважинам District 1: 3,359,141,114


**Вывод:** Функция работает и применима для расчета рисков и прибыли.

### Расчет рисков и прибыли для каждого региона ###

- Создадим 1000 случайных выборок из прогнозируемых моделью значений запасов для 500 скважин. Выберем 200 топовых из них и посчитаем прибыль по формуле (revenue).
- В итоге получим 1000 возможных значений прибили, для которых можно будет найти доверительный интервал и зоны получения убытков (отрицательной прибыли).

In [18]:
def risks(predictions, target, wells_number):
    
    state = np.random.RandomState(42)            #задаем рандом для каждой итерации

    revenue_values = []                          #собираем список из прогнозируемой прибыли

    for i in range(1000):
        predictions_subsumple = predictions.sample(n=wells_number,         #прогноз запасов по модели для занного кол-ва скважин
                                               random_state=state,         #рандом, каждый раз новое значение
                                               replace=True                #возможно повторение значений в выборке
                                              )
        target_subsumple = target[predictions_subsumple.index]             #фактические запасы для отобранных по прогнозам
    
        revenue_values.append(profit(predictions_subsumple, target_subsumple, wells_top)) #собираем список из 1000 значений
    
    revenue_values = pd.Series(revenue_values)
    
    mean =  round(revenue_values.mean(),0)                                 #расчет среднего значения прибыли из 1000 выборок    
    lower = round(revenue_values.quantile(0.05),0)                         #расчет 5% персентиля   
    upper = round(revenue_values.quantile(1-0.05),0)                       #расчет 95% персентиля   
    risk = (revenue_values<0).mean()                                       #расчет количества отрицательных значений к общему количеству
    
    return '{0:,}'.format(mean), '{0:,}'.format(lower), '{0:,}'.format(upper), '{:.1%}'.format(risk)

In [19]:
mean_1, lower_1, upper_1, risk_1 = risks(region_1_predictions, region_1_target, wells_number)

In [20]:
print('\033[1m' + "District 1:"+ '\033[0m')
print("Среднее значение прибыли = ", mean_1)
print("Квантиль 5% = ", lower_1)
print("Квантиль 95% = ", upper_1)
print("Риск убытка = ", risk_1)

District 1:
Среднее значение прибыли =  427,847,560.0
Квантиль 5% =  -9,708,119.0
Квантиль 95% =  865,127,641.0
Риск убытка =  5.5%


In [21]:
mean_2, lower_2, upper_2, risk_2 = risks(region_2_predictions, region_2_target, wells_number)

In [22]:
print('\033[1m' + "District 2:"+ '\033[0m')
print("Среднее значение прибыли = ", mean_2)
print("Квантиль 5% = ", lower_2)
print("Квантиль 95% = ", upper_2)
print("Риск убытка = ", risk_2)

District 2:
Среднее значение прибыли =  511,362,776.0
Квантиль 5% =  156,635,634.0
Квантиль 95% =  868,428,755.0
Риск убытка =  0.9%


In [23]:
mean_3, lower_3, upper_3, risk_3 = risks(region_3_predictions, region_3_target, wells_number)

In [24]:
print('\033[1m' + "District 3:"+ '\033[0m')
print("Среднее значение прибыли = ", mean_3)
print("Квантиль 5% = ", lower_3)
print("Квантиль 95% = ", upper_3)
print("Риск убытка = ", risk_3)

District 3:
Среднее значение прибыли =  402,575,607.0
Квантиль 5% =  -79,364,961.0
Квантиль 95% =  850,883,463.0
Риск убытка =  7.4%


**Вывод:**
- Только в District 2 вероятность убытков меньше 2.5%, кроме того модель очень корошо предсказывает запасы нефти на скважинах из-за сильной линейной связи между признаком f2 и целевым признаком (возможно ошибка или утечка целевого признака).
- До поступления новых данных и уточнения информации по District 2 данный регион рекомендуется к разработке.

## Чек-лист готовности проекта

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован